The aim of the project is to perform analysis of various apps available through Apple store and Google play. <b>The scope is to analysze data points for all free and english apps only.

In [1]:
def explore_data(dataset, start, end, rows_and_columns=True):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

In [2]:
open_Apple_file = open('AppleStore.csv')
open_google_file = open('googleplaystore.csv')
from csv import reader
read_apple = reader(open_Apple_file)
read_google = reader(open_google_file)
apple_list = list(read_apple)
google_list = list(read_google)
explore_data(apple_list,0,5)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


Number of rows: 7198
Number of columns: 16


In the code below we are analyzing the whether all the records in the csv file match with the number of columns mentioned in the header (first row). We are then identifying the rows where the row length does not match with the header length.

In [3]:
header_length = len(google_list[0])
print('google header length',header_length)
for row in google_list[1:]:
    row_length = len(row)
    if row_length != header_length:
        print('google',google_list.index(row))
        print(row_length)

header_length_apple = len(apple_list[0])
print('apple header length',header_length_apple)
for row in apple_list[1:]:
    row_length = len(row)
    if row_length != header_length_apple:
        print('apple',apple_list.index(row))
        print(row_length)

google header length 13
google 10473
12
apple header length 16


We are then deleting the record which is not having the same number of columns as present in the header row.

In [4]:
del google_list[10473]

There is a possibility of having duplicate records in both of the csv files. We will now identify the duplicate records present in the google csv file

In [5]:
unique_apps = []
duplicate_apps = []
for row in google_list[1:]:
    app_name = row[0]
    if app_name in unique_apps:
        duplicate_apps.append(app_name)
    else:
        unique_apps.append(app_name)
#print(duplicate_apps[0:15])
print('Number of duplicate apps google ', len(duplicate_apps))

unique_apps_apple = []
duplicate_apps_apple = []
for row in apple_list[1:]:
    app_name = row[1]
    if app_name in unique_apps_apple:
        duplicate_apps_apple.append(app_name)
    else:
        unique_apps_apple.append(app_name)
#print(duplicate_apps_apple[0:15])
print('Number of duplicate apps apple ', len(duplicate_apps_apple))


Number of duplicate apps google  1181
Number of duplicate apps apple  2


Now that we have identified that there are duplicate records, we need to define a criteria to remove duplicate records. Looking closely at the header columns, we find that there is a column at ideex 5 called 'rating_count_tot'. Out of the duplicate records for a given app, we will retain the record having highest number for 'rating_count_tot'. Higher the number of reviews, the more recent the data should be.

Given below is an example for app 'Box'

In [6]:
for row in google_list[1:]:
    app_name = row[0]
    if app_name == 'Box':
        print(row)

['Box', 'BUSINESS', '4.2', '159872', 'Varies with device', '10,000,000+', 'Free', '0', 'Everyone', 'Business', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Box', 'BUSINESS', '4.2', '159872', 'Varies with device', '10,000,000+', 'Free', '0', 'Everyone', 'Business', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Box', 'BUSINESS', '4.2', '159872', 'Varies with device', '10,000,000+', 'Free', '0', 'Everyone', 'Business', 'July 31, 2018', 'Varies with device', 'Varies with device']


In the above example, we find that the number of reviews are same for all the three records identified for app 'Box'. In such case we will retain only one record for the app and delete the remaining two records.

Now we will create a dictionary where easch key is a unique app name and the corresponding dictionary value is the highest number of reviews for that app.

In [7]:
reviews_max = {}
for row in google_list[1:]:
    name = row[0]
    n_reviews = row[3]
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    if name not in reviews_max:
         reviews_max[name] = n_reviews
print('Length of the google dictionary is',len(reviews_max))

reviews_max_apple = {}
for row in apple_list[1:]:
    apple_name = row[1]
    n_reviews_apple = row[5]
    if apple_name in reviews_max_apple and reviews_max_apple[apple_name] < n_reviews_apple:
        reviews_max_apple[apple_name] = n_reviews_apple
    if apple_name not in reviews_max_apple:
         reviews_max_apple[apple_name] = n_reviews_apple
print('Length of the apple dictionary is',len(reviews_max_apple))
#print(reviews_max_apple)

Length of the google dictionary is 9659
Length of the apple dictionary is 7195


Now we will use this dictionay to remove the duplicate records from google list.
We will create two empty lists.
1. 'android_clean' which will store our cleaned data set
2. 'already_added' which will just store app names - This is needed to avoid duplicate addition of app details to 'android_clean' list since there are some apps such as 'Box' which have identical duplicate records - including the number of reviews.

In [8]:
android_clean = []
already_added = []
for row in google_list[1:]:
    name = row[0]
    n_reviews = float(row[3])
    #print(n_reviews == reviews_max[name])
    if n_reviews == float(reviews_max[name]) and name not in already_added:
        #print('I am here')
        android_clean.append(row)
        already_added.append(name)
print('Number of records in android_clean is', len(android_clean))
print('Number of records in already_added is', len(already_added))

apple_clean = []
already_added_apple = []
for row in apple_list[1:]:
    name_apple = row[1]
    n_reviews_apple = float(row[5])
    #print(len(apple_list[1:]))
    #print(n_reviews == reviews_max[name])
    if n_reviews_apple == float(reviews_max_apple[name_apple]) and name_apple not in already_added_apple:
        #print('I am here')
        apple_clean.append(row)
        already_added_apple.append(name_apple)
print('Number of records in apple_clean is', len(apple_clean))
print('Number of records in already_added_apple is', len(already_added_apple))

Number of records in android_clean is 9659
Number of records in already_added is 9659
Number of records in apple_clean is 7195
Number of records in already_added_apple is 7195


If you observe our datasets closely, you will find that there are records for non-english apps as well. This project focuses on analysis of english apps only. Hence we will now find a way to identify and delete the records for non-english apps.

English characters have unique numbers associated with them in the range of 0 to 127 (according to ASCII). Hence we will now look at app names in our data sets and identiy all such records where app name has characters which are not in the range of 0 to 127

Given below is a function which takes string as an inputs and returns 'False' if there is any character in the string that doesn't belong to the set of common English characters, otherwise it returns 'True'

In [9]:
def isEnglish(a_string):
    for character in a_string:
        if 0 <= ord(character) <= 127:
            return True
        else:
            return False
print(isEnglish('爱'))
    

False


The above function will not be able to handle emojis and special characters which fall outside the ASCII range. Hence if we use this function as it is, we will incorrectly lable apps as "Non-English" if the app name contains emojis or special characters. 

To minimize the impact of data loss, we'll only remove an app if its name has more than three characters with corresponding numbers falling outside the ASCII range. This means all English apps with up to three emoji or other special characters will still be labeled as English. Our filter function is still not perfect, but it should be fairly effective.

In [14]:
def isEnglishSpecial(a_string):
    special_character_count = 0
    for character in a_string:
        if not(0 <= ord(character) <= 127):
            special_character_count += 1
    
    if special_character_count > 3:
        return False
    else:
        return True
    
#print(isEnglishSpecial('爱奇艺PPS -《欢乐颂2》电视剧热播'))

Now we will use this new function to filter out non-English apps from both data sets - 'android_clean' and 'already_added'

In [11]:
android_final = []
already_added_final = []
for row in android_clean:
    if isEnglishSpecial(row[0]):
        android_final.append(row)

for row in already_added:
    if isEnglishSpecial(row):
        already_added_final.append(row)

print('Number of records in android_final are',len(android_final))
#print('Number of records in already_added_final are',len(already_added_final))

ios_final = []
for row in apple_clean:
    if isEnglishSpecial(row[1]):
        ios_final.append(row)
print('Number of records in ios_final are',len(ios_final))

Number of records in android_final are 9614
Number of records in ios_final are 6181


As we mentioned in the introduction, we will analyze only those apps that are free to download and install. Our data sets contain both free and non-free apps; we'll need to isolate only the free apps for our analysis.

We will now filter out the non-free apps from the datasets - 'android_final'

In [13]:
android_free_final = []
for row in android_final:
#print(row)
    price = row[7]
    if price == '0':
        android_free_final.append(row)
    

print('Number of records in android_free_final are',len(android_free_final))

ios_free_final = []
for row in ios_final:
#print(row)
    price = row[4]
    if price == '0.0':
        ios_free_final.append(row)
    

print('Number of records in ios_free_final are',len(ios_free_final))


Number of records in android_free_final are 8862
Number of records in ios_free_final are 3220
